In [1]:
from tichandlers import SBtickethandler
from tichandlers import FGtickethandler
from tichandlers import MTtickethandler
from tichandlers import MAtickethandler
from tichandlers import RBMRtickethandler
from tichandlers import FVtickethandler

In [2]:
from timelinebuilding import ticketgenerator
from timelinebuilding import addtotimeline
from timelinebuilding import travel
from timelinebuilding import ticketgenerator
from timelinebuilding import nexttau
from timelinebuilding import futurebuilder
from timelinebuilding import femaleprobs


In [3]:
o_t=ticketgenerator(1,0,0,-1,0,{})
new_ac=-2
new_targ=0
timeline=[]
futurebuilder(o_t, new_ac, new_targ)

futurebuilder(timeline[0], -1, new_targ)
print(timeline)

IndexError: list index out of range

In [ ]:
#Bowerbird null model (all males are guarders)

#Key
#Notes- Paramters contain underscores, while functions don't
#SB: Stay at bower
#FG: Foraging
#MT: Maraud travel (travel to rival's bower)
#MA: Maraud action (destroy bower if absent, have antagonistic interaction in present)
#MR: Maraud return (return to own bower)
#RB: Repair own bower
#MT_vs_FG: the probability of next transition away from SB being to MT (otherwise it would be to FG)
#a=-1: denotes a staying at bower action on ticket
#a=-2: denotes a foraging action on ticket
#a=-3: denotes a female visiting action on a ticket
#a=-4: denotes travel to another male's bower
#a=-5: denotes marauding of another male's bower
#a=-6: denotes returning from another male's bower
#a=-7: denotes a bower repair action on a ticket
#target: the male/node an action is preformed on. For maurading, the target is the bird under potential attack. For mating, target is male being mated with. For other activities other then maurading and mating, owner and target are the same.
#owner: The male or female actively performing an action. For mating, the female is the owner. For all other activities, the owner is male.
#fullyconnectednetworkwriter: generates network where every male is connected
#addtotimeline: adds tickets to timeline
#travel_times_linear_p: generates a matrix that contains the probabilities of travelling to a given a bower 
#if the relationship between distance and travel preference is linear
#improb: The probability of traveling improb_distance or less
#improb_distance: The distance at which there is only a (1-improb)% chance of choosing to travel
#lamb: lambda calculated by solving improb=1-numpy.exp(-lamb*improb_distance)
#bower_states: options are 1 (bower intact) and 0 (bower destroyed-will never be the case when all guarders) and another value for time that mating will end (assuming mating is happening)
#male_states: options are [leave_time, leave_action, SB_timeholder] (male present at bower) and 0 (male absent from bower)
#fitness_states: keeps track of number of matings a given male has had
#t: time
#nodes: total number of male bowerbirds in the network
#node_dist: the node by nde matrix with distances between each node
#node_graph: matrix with individuals 1 through nodes numbered by going through each row
#tau: Length of time an activity takes. If a foraging ticket has a tau of 0.5 and occurs at t=1, then the bird is foraging for 30 minutes, at which point a SB activity will start (at t=1.5).
#t_court: the amount of time it takes a female to court+mate with a male, generated from a normal distribution


import math
import random
import numpy as numpy
import matplotlib.pyplot as plt

from timelinebuilding import ticketgenerator
from timelinebuilding import addtotimeline
from timelinebuilding import travel
from timelinebuilding import ticketgenerator
from timelinebuilding import nexttau
from timelinebuilding import futurebuilder
from timelinebuilding import femaleprobs

#Parameters:
t=0.0 #start at time 0
t_max=12#12*4*30 #2 months in hrs (assume no nights so 12 hr = 1 day)
timeholder=0
timeline=[] #initialize the timeline
discards=[] #where we put the old version of tickets that get over-written/moved
#We've moved away from using these params with expovariate (how we handle FV is still TBD)
#So THE BELOW 3 SHOULD BE COMMENTED OUT (but rn we don't want the handler funcs to yell at us)
FV_param=[1, t_max/2] #5days, tmax/3 #0.2 #nodes/0.19 #totally random
FG_param=3 #larger values lead to foraging ending earlier, because taus are smaller
SB_param=1.5 #larger values lead to bowerstay ending earlier, because taus are smaller
RB_param=2 #made up -- takes 30mins (every time -- no distribution) to repair bower
MA_param=.1 #made up -- takes 6mins (every time -- no distribution) to maraud bower
MT_vs_FG=.05 #compared 20 feedings/day to .1 maraudings/hr
nodes=4
bower_states=[1]*nodes
male_states=[0]*nodes #start at foraging now
mating_states=[0]*nodes
fitness_states=[0]*nodes
success_rates=1
success_times=[] #going to change this to be a vector of times when the female leaves (at this point, sucessful mating has occured)
recents_list=[]
max_visits=6 #STEEEEVEEEE
bird_speed=12*3600 #m/hr (12m/s)
d=150.0

In [ ]:
#Seed the timeline

d={}

#changed so they start out foraging not SB!
for i in range(nodes):
    init_FG_time=t+nexttau(-2, i, i, 0)
    init_FG_tic=ticketgenerator(init_FG_time-t, t, i, -2, i, {})
    addtotimeline(init_FG_tic, timeline)


#the logic with ts and taus is not parallel to all other ticket types, but it seems to work... maybe change later    
for i in range(int(3*nodes)): #
    first_male=random.choice(list(range(nodes))) #The lucky bowerbird that is chosen by the female first
    first_female_time=0 #determine the time when the first female arrives
    first_female_tic=ticketgenerator(0,first_female_time, i, -3, first_male, {}) #first female ticket
    addtotimeline(first_female_tic, timeline)
    d["rl{0}".format(i)]=[]
    
        
    


In [ ]:
#read tickets, generate more tickets, and move through the timeline until t_max is reached

while t<t_max:
    if timeholder>(len(timeline)-1): #if timeholder exceeds max timeline index
        print('end')
        break
    next_tic=timeline[timeholder] #look at the ticket corresponding to where we are on the timeline
    action=next_tic['action']
    ow=next_tic['owner']
    if action==-1:
        SBtickethandler(next_tic)
    elif action==-2:
        FGtickethandler(next_tic)
    elif action==-3:
        d["rl{0}".format(ow)]=FVtickethandler(next_tic, d["rl{0}".format(ow)])
    elif action==-4:
        MTtickethandler(next_tic)
    elif action==-5:
        MAtickethandler(next_tic)
    else:
        RBMRtickethandler(next_tic) 
    t=next_tic['time'] #new time based on the ticket we just read
    timeholder=timeholder+1 #increment timeholder



In [ ]:
d={}

for i in range(nodes):
    d["tl{0}".format(i)]=[]
for j in range(len(timeline[0:timeholder])):
    #for i in range(nodes):
     #   if timeline[j]['owner']==i:
    if timeline[j]['action']==-3:
        d["tl{0}".format(timeline[j]['target'])].append(timeline[j])  
    else:
        d["tl{0}".format(timeline[j]['owner'])].append(timeline[j])

print("tl0----")
print(d["tl0"])
print("tl1----")
print(d["tl1"])
print("tl2----")
print(d["tl2"])
print("tl3----")
print(d["tl3"])


for i in range(nodes):
    tl=d["tl{0}".format(i)]
    l=len(tl)
    ta=[0]
    fem=[]
    for j in range(l):
        if tl[j]['action']!=-3: #female visit taus don't mean anything for the male bc she isn't necessarily coming back to the same male
            ta.append(tl[j]['time'])
        else:
            fem.append(tl[j]['time'])
    d["times{0}".format(i)]=ta
    d["fvtimes{0}".format(i)]=fem

#print("times0----")
#print(d["times0"])
#print("times1----")
#print(d["times1"])
#print("times2----")
#print(d["times2"])
#print("times3----")
#print(d["times3"]) 

#append tmax - sumoftaus to each list (so it reaches tmax exactly)
for i in range(nodes):
    d["times{0}".format(i)].append(t_max)


import csv
timess=[d["times0"],d["times1"],d["times2"],d["times3"]]
csvfile = "example_times2"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(timess)
femtimess=[]
for i in range(nodes):
    femtimess.append(d["fvtimes{0}".format(i)])

csvfile = "example_fvtimes"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(femtimess)

           
    

In [ ]:
#PERCENTS NEED TO ADD UP

#Percentage times at bower not currently correct-- based on all guard!!!!
#Will recalculate in the future after changing mating to not be instantaneous 
for i in range(nodes):
    tl=d["tl{0}".format(i)]
    l=len(tl)
    ta=[]
    accSB=0
    accFG=0
    accM=0
    accRB=0
    allacc=0
    FV_list=[]
    for j in range(l):
        if tl[j]['action']!=-3: #female visit taus don't mean anything for the male bc she isn't necessarily coming back to the same male
            ta.append(tl[j]['tau'])
            #print("time according to taus is {:f}".format(allacc))
            #print("time according to tickets is {:f}".format(tl[j]['time']))
            allacc+=tl[j]['tau']
            if tl[j]['action']==-1:
                accSB=accSB+tl[j]['tau']
                #print("action: {:d}, tau: {:f}".format(tl[j]['action'], tl[j]['tau']))
            elif tl[j]['action']==-2:
                accFG=accFG+tl[j]['tau']
                #print("action: {:d}, tau: {:f}".format(tl[j]['action'], tl[j]['tau']))
            elif tl[j]['action']==-7:
                accRB=accRB+tl[j]['tau']
                #print("action: {:d}, tau: {:f}".format(tl[j]['action'], tl[j]['tau']))
            elif tl[j]['action']==-4 or tl[j]['action']==-5 or tl[j]['action']==-6:
                accM=accM+tl[j]['tau']  
                #print("action: {:d}, tau: {:f}".format(tl[j]['action'], tl[j]['tau']))
        else:
            #print("FV")
            FV_list.append(tl[j])
    d["taus{0}".format(i)]=ta
    su=sum([accSB/t_max, accM/t_max, accFG/t_max, accRB/t_max])
    if su==1:
        print("WE GOOD!: {:f}".format(su))
    else:
        print("NOT GOOD!: {:f}".format(su))
    print("Bird {:d}'s % time at bower is {:f}".format(i, accSB/t_max)) 
    print("Bird {:d}'s % time marauding is {:f}".format(i, accM/t_max)) 
    print("Bird {:d}'s % time foraging is {:f}".format(i, accFG/t_max)) 
    print("Bird {:d}'s % time repairing bower is {:f}".format(i, accRB/t_max)) 
    print("Bird {:d}'s separate accs sum to {:f}".format(i,(accSB+accM+accFG+accRB)))
    print("Bird {:d}'s allacc is {:f}".format(i,allacc))
    
#append tmax - sumoftaus to each list (so it reaches tmax exactly) --IS THIS EVEN OK?
#for i in range(nodes):
#    d["taus{0}".format(i)].append(t_max-sum(d["taus{0}".format(i)]))
    
    
print("taus0 sum----")
#print(d["taus0"])
print(sum(d["taus0"]))
print("taus1 sum----")
#print(d["taus1"])
print(sum(d["taus1"]))
print("taus2 sum----")
#print(d["taus2"])
print(sum(d["taus2"]))
print("taus3 sum----")
#print(d["taus3"]) 
print(sum(d["taus3"]))


    


In [ ]:
print(success_times)
print(len(success_times))

In [ ]:
plt.hist(success_times)
plt.show()

In [ ]:
print(len(femtimess))
#print(femtimess)
fts = [item for sublist in femtimess for item in sublist]
print(len(fts))

In [ ]:
print(len(discards))
print(discards)

In [ ]:
fitness_states

In [ ]:
nice_timeline=[]
for i in range(timeholder):
    tic=timeline[i]
    t=tic['time']
    ta=tic['tau']
    ow=tic['owner']
    ac=tic['action']
    targ=['target']
    nice_timeline[i]={
        'tau': ta,
        'time': t,
        'owner': ow,
        'action': ac,
        'targ': targ
    }
    {'time': tic['time'], 'action': tic['']}
    switcher = { #NOTE: used abs
        -1: print("{:f} -- male {:d} stays at bower for {:f}hrs".format(t,ow,ta)),
        -2: print("{:f} -- male {:d} forages for {:f}hrs".format(t,ow,ta)),
        -3: print("{:f} -- female {:d} visits male {:d} for {:f}hrs".format(t,ow,targ,ta)),
        -4: print("{:f} -- male {:d} travels to male {:d}'s bower for {:f}hrs".format(t,ow,targ,ta)),
        -5: print("{:f} -- male {:d} visits male {:d}'s bower for {:f}hrs".format(t,ow,targ,ta))
        -6: print("{:f} -- male {:d} returns from male {:d}'s bower for {:f}hrs".format(t,ow,targ,ta))
        -7: print("{:f} -- male {:d} repairs his bower for {:f}hrs".format(t,ow,targ,ta)) 
    }
    switcher.get(action)

    
#GOES BELOW WHILE LOOP